<a href="https://colab.research.google.com/github/Reuben-Stephen-John/Classification-Models/blob/main/Multi_level_Classification_Virus_Varient_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import sklearn as sk
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c gdsc-data-science-recruitments-2022
!unzip gdsc-data-science-recruitments-2022.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open gdsc-data-science-recruitments-2022.zip, gdsc-data-science-recruitments-2022.zip.zip or gdsc-data-science-recruitments-2022.zip.ZIP.


# New Section

In [ ]:
training= pd.read_csv("Train.csv")
training

FileNotFoundError: ignored

In [ ]:
training.describe()

In [ ]:
training.info()

In [ ]:
training.drop("Pre-existing symptoms",inplace=True,axis=1)#since there are many na (unfilled)values
training["Headache"].fillna(training['Headache'].mode().values[0],inplace = True)#replacing na values
training['Nausea'].fillna(training['Nausea'].mode().values[0],inplace = True)#replacing na values
training.dropna(inplace=True,axis=0)#removing na rows since they wont accept much 
training['family history'] = training['family history'].replace("Yes",1)#one hot encoding-textual data to numberical data
training['family history'] = training['family history'].replace("No",0)#one hot encoding

In [ ]:
training.info()

In [ ]:
training.isnull().sum()

In [ ]:
def makeCountPlot(data, width=0, height=4, hue=None):
  if width == 0:
    width = len(data.unique())*1.25
  plt.figure(figsize=(width, height))
  sns.countplot(x=data, hue=hue)

In [ ]:
makeCountPlot(data=training['Variant'])


In [ ]:
for i in training.columns[1:-2]:
  makeCountPlot(data=training[i],hue=training["Variant"])


In [ ]:
#graphs plotted show the different relations between varient, their count and the level of symptoms

In [ ]:
training.isnull().info()

In [ ]:
training

In [ ]:
training.describe()

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(training.corr(),annot=True)#Correlation Matrix for each coloumn 

In [ ]:
training.corr()

In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(24,)),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(6,activation='softmax'),
])
#creating a model and optimizing it 
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])



In [ ]:
train_labels = {}
for i in range(0,6):
  key = "Variant "+str(i+1)
  train_labels[key] = i
print(train_labels)
for i in train_labels:
  training['Variant'] = training['Variant'].replace(i,train_labels[i])
training

In [ ]:
training = pd.get_dummies(training,columns = ['Region'])
training

In [ ]:
variant = training['Variant']
variant
training.drop('Variant',axis = 1,inplace = True)

In [ ]:
training.info()

In [ ]:
training

In [ ]:
x = training.iloc[:,1:]#weighted sums
x = np.array(x)
x_norm = x/x.max(axis=0)
x_norm

In [ ]:
model.fit(x_norm,variant,epochs = 90)#running the model

In [ ]:
test= pd.read_csv("Test.csv")#taking the test data and runnning the same data cleaning process
test

In [ ]:
test.drop("Pre-existing symptoms",inplace=True,axis=1)
test = pd.get_dummies(test,columns = ['Region'])

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

In [ ]:
test['family history'] = test['family history'].replace("Yes",1)
test['family history'] = test['family history'].replace("No",0)

In [ ]:
test.info()

In [ ]:
test1=test.iloc[:,1:]
x1 = np.array(test1)
test_norm = x1/x1.max(axis=0)
test_norm
test_norm.shape

In [ ]:
predictions=model.predict(test_norm)

In [ ]:
pred=[]
labels=["Variant 1","Variant 2","Variant 3","Variant 4","Variant 5","Variant 6"]
for i in predictions:
    pred.append(labels[np.argmax(i)])



In [ ]:
def unique(list1):
  
    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    for x in unique_list:
        print(x)
unique(pred)

In [ ]:
data = {'ID':test["ID"],'Variant':pred}
df=pd.DataFrame(data)
df
df.to_csv('Submission.csv', index=False)


In [ ]:
okay=pd.read_csv("Submission.csv")
okay